In [ ]:
# 괌 날씨 데이터 분석 - Part 1: CSV 데이터 분석

이 노트북에서는 2020년부터 2022년까지의 괌 날씨 CSV 데이터를 분석하고 시각화합니다.

## 목차
1. 데이터 로딩 및 탐색
2. 데이터 전처리
3. 연도별/월별 분석
4. 시각화
5. 인사이트 도출


In [ ]:
# 필요한 라이브러리 import
import sys
import os
sys.path.append('../src')  # src 디렉토리를 Python 경로에 추가

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 우리가 만든 모듈들 import
from data_processing import load_all_csv_files, preprocess_weather_data, get_yearly_summary, get_monthly_summary
from visualization import plot_monthly_temperature_by_year, plot_monthly_precipitation_by_year, plot_yearly_summary, plot_temperature_departure, create_comprehensive_dashboard

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 import 완료!")


In [ ]:
## 1. 데이터 로딩 및 탐색


In [ ]:
# 1.1 CSV 파일들 로딩
combined_df, csv_files = load_all_csv_files("../data")

print(f"총 {len(csv_files)}개의 CSV 파일을 로딩했습니다.")
print(f"결합된 데이터프레임 크기: {combined_df.shape}")
print("\n파일 목록:")
for i, file in enumerate(csv_files, 1):
    print(f"{i:2d}. {file}")

print(f"\n데이터 기간: {combined_df['Date'].min()} ~ {combined_df['Date'].max()}")
print(f"총 {len(combined_df)} 일의 데이터")


In [ ]:
# 1.2 데이터 구조 탐색
print("=== 데이터 기본 정보 ===")
combined_df.info()

print("\n=== 첫 5행 데이터 ===")
print(combined_df.head())

print("\n=== 기본 통계 ===")
print(combined_df.describe())


In [ ]:
# 1.3 특수 값들 확인
print("=== Precipitation 컬럼의 고유값 ===")
print(f"고유값: {sorted(combined_df['Precipitation'].unique())}")
print(f"'T' 값의 개수: {sum(combined_df['Precipitation'] == 'T')}")

print("\n=== Snow Depth 컬럼의 고유값 ===")
print(f"고유값: {sorted(combined_df['Snow Depth'].unique())}")
print(f"'M' 값의 개수: {sum(combined_df['Snow Depth'] == 'M')}")

print("\n참고:")
print("- 'T' (Trace): 강수량이 측정 가능한 최소량보다 적음을 의미")
print("- 'M' (Missing): 결측값을 의미")


In [ ]:
## 2. 데이터 전처리

데이터를 분석하기 전에 다음 전처리 작업을 수행합니다:
1. Date 컬럼을 datetime 타입으로 변환
2. Year, Month, Day 컬럼 추가
3. Precipitation의 'T' → 0 변환
4. Snow Depth의 'M' → 0 변환
5. 화씨 → 섭씨 변환 (Maximum, Minimum, Average)
6. 인치 → mm 변환 (Precipitation, Snow Depth)


In [ ]:
# 2.1 데이터 전처리 실행
print("데이터 전처리 중...")
processed_df = preprocess_weather_data(combined_df)

print("전처리 완료!")
print(f"전처리 후 데이터 크기: {processed_df.shape}")

print("\n=== 전처리 후 데이터 타입 ===")
processed_df.info()

print("\n=== 전처리 후 첫 5행 ===")
print(processed_df.head())


In [ ]:
# 2.2 전처리 결과 확인
print("=== 기온 변환 결과 (화씨 → 섭씨) ===")
print("샘플 데이터 (첫 5행):")
print(processed_df[['Date', 'Maximum', 'Minimum', 'Average']].head())

print(f"\n평균 기온 범위: {processed_df['Average'].min():.1f}°C ~ {processed_df['Average'].max():.1f}°C")
print(f"최고 기온 범위: {processed_df['Maximum'].min():.1f}°C ~ {processed_df['Maximum'].max():.1f}°C")
print(f"최저 기온 범위: {processed_df['Minimum'].min():.1f}°C ~ {processed_df['Minimum'].max():.1f}°C")

print("\n=== 강수량 변환 결과 (인치 → mm) ===")
print(f"강수량 범위: {processed_df['Precipitation'].min():.1f}mm ~ {processed_df['Precipitation'].max():.1f}mm")
print(f"평균 일일 강수량: {processed_df['Precipitation'].mean():.2f}mm")

# 전처리된 데이터를 CSV로 저장
processed_df.to_csv("../results/processed_weather_data.csv", index=False)
print("\n전처리된 데이터를 'results/processed_weather_data.csv'에 저장했습니다.")


In [ ]:
## 3. 연도별/월별 분석


In [ ]:
# 3.1 연도별 요약 통계
yearly_summary = get_yearly_summary(processed_df)
print("=== 연도별 요약 통계 ===")
print(yearly_summary.round(2))

print("\n=== 연도별 특징 ===")
for _, row in yearly_summary.iterrows():
    year = int(row['Year'])
    avg_temp = row['Average']
    total_precip = row['Precipitation']
    departure = row['Departure']
    
    print(f"\n{year}년:")
    print(f"  • 평균 기온: {avg_temp:.1f}°C")
    print(f"  • 총 강수량: {total_precip:.0f}mm")
    print(f"  • 평년 대비: {departure:+.1f}°C ({'평년보다 높음' if departure > 0 else '평년보다 낮음' if departure < 0 else '평년과 비슷'})")


In [ ]:
# 3.2 월별 분석 (전체 기간 통합)
monthly_overall = processed_df.groupby('Month').agg({
    'Average': 'mean',
    'Maximum': 'mean',
    'Minimum': 'mean',
    'Precipitation': 'mean',
    'Departure': 'mean'
}).round(2)

print("=== 월별 평균 (2020-2022년 통합) ===")
print(monthly_overall)

# 가장 덥고 추운 달 찾기
hottest_month = monthly_overall['Average'].idxmax()
coldest_month = monthly_overall['Average'].idxmin()
wettest_month = monthly_overall['Precipitation'].idxmax()
driest_month = monthly_overall['Precipitation'].idxmin()

month_names = ['', '1월', '2월', '3월', '4월', '5월', '6월', 
               '7월', '8월', '9월', '10월', '11월', '12월']

print(f"\n=== 극값 분석 ===")
print(f"가장 더운 달: {month_names[hottest_month]} ({monthly_overall.loc[hottest_month, 'Average']:.1f}°C)")
print(f"가장 추운 달: {month_names[coldest_month]} ({monthly_overall.loc[coldest_month, 'Average']:.1f}°C)")
print(f"가장 비가 많은 달: {month_names[wettest_month]} ({monthly_overall.loc[wettest_month, 'Precipitation']:.1f}mm)")
print(f"가장 건조한 달: {month_names[driest_month]} ({monthly_overall.loc[driest_month, 'Precipitation']:.1f}mm)")


In [ ]:
## 4. 데이터 시각화

다양한 그래프를 통해 괌의 날씨 패턴을 시각적으로 분석합니다.


In [ ]:
# 4.1 연도별 월평균 기온 그래프
print("연도별 월평균 기온 그래프를 생성 중...")
plot_monthly_temperature_by_year(processed_df, save_path="../results/monthly_temperature_by_year.png")


In [ ]:
# 4.2 연도별 월평균 강수량 그래프
print("연도별 월평균 강수량 그래프를 생성 중...")
plot_monthly_precipitation_by_year(processed_df, save_path="../results/monthly_precipitation_by_year.png")


In [ ]:
# 4.3 연간 요약 그래프 (평균 기온 & 총 강수량)
print("연간 요약 그래프를 생성 중...")
plot_yearly_summary(processed_df, save_path="../results/yearly_summary.png")


In [ ]:
# 4.4 월별 기온 편차 그래프
print("월별 기온 편차 그래프를 생성 중...")
plot_temperature_departure(processed_df, save_path="../results/temperature_departure.png")


In [ ]:
# 4.5 종합 대시보드
print("종합 대시보드를 생성 중...")
create_comprehensive_dashboard(processed_df, save_path="../results/comprehensive_dashboard.png")


In [ ]:
## 5. 인사이트 및 결론


In [ ]:
# 5.1 주요 인사이트 분석
print("=== 괌 날씨 데이터 분석 결과 (2020-2022) ===\n")

# 기온 안정성 분석
temp_std = processed_df.groupby('Year')['Average'].std()
print("🌡️ 기온의 안정성:")
print(f"   연도별 일평균 기온의 표준편차: {temp_std.mean():.2f}°C")
print(f"   → 괌은 연중 매우 안정적인 기온을 유지 (열대 해양성 기후 특성)")

# 계절성 분석  
summer_months = [6, 7, 8, 9]  # 여름철 (우기)
winter_months = [12, 1, 2, 3]  # 건기

summer_avg = processed_df[processed_df['Month'].isin(summer_months)]['Average'].mean()
winter_avg = processed_df[processed_df['Month'].isin(winter_months)]['Average'].mean()
temp_diff = summer_avg - winter_avg

print(f"\n🌴 계절별 특성:")
print(f"   여름철(6-9월) 평균 기온: {summer_avg:.1f}°C")
print(f"   건기(12-3월) 평균 기온: {winter_avg:.1f}°C") 
print(f"   계절간 기온 차이: {temp_diff:.1f}°C (매우 작음)")

# 강수량 변동성 분석
annual_precip = processed_df.groupby('Year')['Precipitation'].sum()
precip_variation = annual_precip.std() / annual_precip.mean() * 100

print(f"\n🌧️ 강수량 특성:")
print(f"   연간 총 강수량 평균: {annual_precip.mean():.0f}mm")
print(f"   연간 강수량 변동계수: {precip_variation:.1f}%")
if precip_variation < 20:
    print(f"   → 상대적으로 안정적인 강수 패턴")
else:
    print(f"   → 연도별 강수량 변동이 큼")

# 극값 분석
extreme_hot = processed_df[processed_df['Maximum'] > processed_df['Maximum'].quantile(0.95)]
extreme_cold = processed_df[processed_df['Minimum'] < processed_df['Minimum'].quantile(0.05)]

print(f"\n🔥 극값 분석:")
print(f"   최고 기온 기록: {processed_df['Maximum'].max():.1f}°C")
print(f"   최저 기온 기록: {processed_df['Minimum'].min():.1f}°C")
print(f"   일교차 평균: {(processed_df['Maximum'] - processed_df['Minimum']).mean():.1f}°C")

# 기후 변화 신호
departure_trend = processed_df.groupby('Year')['Departure'].mean()
print(f"\n📈 기후 변화 신호:")
for year, dept in departure_trend.items():
    trend = "상승" if dept > 0 else "하락" if dept < 0 else "평년 수준"
    print(f"   {year}년 평년 대비: {dept:+.2f}°C ({trend})")

if departure_trend.iloc[-1] > departure_trend.iloc[0]:
    print("   → 최근 평년 대비 기온 상승 경향")
else:
    print("   → 최근 평년 대비 기온 하락 또는 안정")
